# Laboratoire 1 - Recommandations par filtres collaboratifs user-user et item-item

Auteurs:

1974328 - Rocca Baptiste

1527096 - Dos Santos Alexandre

In [ ]:
!pip install --user pandas
!pip install --user scipy
!pip install --user scikit-learn

import warnings
warnings.filterwarnings("ignore")

In [1]:
import pandas as pd

Nous allons lire les données et en imprimer une partie de manière à mieux appréhender le contenu et ainsi pouvoir les traiter de façon cohérente.

In [20]:
data = pd.read_csv('./u.data.csv', delimiter= '|')
item = pd.read_csv('./u.item.csv', delimiter= '|')
user = pd.read_csv('./u.user.csv', delimiter= '|')

user.columns = [column.replace(" ", "") for column in user.columns];

In [21]:
print(data.head())
print(item.head())
print(user.head())

   user.id  item.id  rating  timestamp
0      196      242       3  881250949
1      186      302       3  891717742
2       22      377       1  878887116
3      244       51       2  880606923
4      166      346       1  886397596
   movie id        movie title   release date    video release date   \
0          1   Toy Story (1995)    01-Jan-1995                   NaN   
1          2   GoldenEye (1995)    01-Jan-1995                   NaN   
2          3  Four Rooms (1995)    01-Jan-1995                   NaN   
3          4  Get Shorty (1995)    01-Jan-1995                   NaN   
4          5     Copycat (1995)    01-Jan-1995                   NaN   

                                           IMDb URL    unknown    Action   \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...          0         0   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...          0         1   
2  http://us.imdb.com/M/title-exact?Four%20Rooms%...          0         0   
3  http://us.imdb.com/M/t

# Question 1 - moyenne par job et par âge

## Moyenne des notes en fonction du job 

In [33]:
joined = data.join(user.set_index('id'), on='user.id')
job_means = pd.DataFrame(joined.groupby(["job"]).mean().loc[:,'rating'])
job_means['counts'] = user.groupby(["job"]).count().loc[:,'id']
print(job_means)

                 rating  counts
job                            
administrator  3.635646      79
artist         3.653380      28
doctor         3.688889       7
educator       3.670621      95
engineer       3.541407      67
entertainment  3.441050      18
executive      3.349104      32
healthcare     2.896220      16
homemaker      3.301003       7
lawyer         3.735316      12
librarian      3.560781      51
marketing      3.485641      26
none           3.779134       9
other          3.552377     105
programmer     3.568260      66
retired        3.466750      14
salesman       3.582944      12
scientist      3.611273      31
student        3.515143     196
technician     3.532230      27
writer         3.375723      45


On peut noter que tous les job semblent posséder environt la même moyenne (avec une moyenne plus basse pour les personne travaillant dans le domaine de la santé qui sont pourtant au nombre de 16)

## Moyenne des notes par age

In [47]:
age_means = pd.DataFrame(joined.groupby(["age"]).mean().loc[:,'rating'])
age_means['counts'] = user.groupby(["age"]).count().loc[:,'id']
print(age_means)

       rating  counts
age                  
7    3.767442       1
10   3.387097       1
11   2.925926       1
13   3.470825       5
14   3.375000       3
15   3.264484       6
16   3.331343       5
17   3.516165      14
18   3.696710      18
19   3.400968      23
20   3.670580      32
21   3.461589      27
22   3.253330      37
23   3.307492      28
24   3.541923      33
25   3.577872      38
26   3.213531      34
27   3.525144      35
28   3.541862      36
29   3.448493      32
30   3.438862      39
31   3.379035      25
32   3.619399      28
33   3.628785      26
34   3.434505      17
35   3.694321      27
36   3.504039      21
37   3.713652      19
38   3.503983      17
39   3.627586      22
..        ...     ...
41   3.661885      10
42   3.432019      21
43   3.628378      13
44   3.566276      23
45   3.727862      15
46   3.765201      12
47   3.480173      14
48   3.588406      20
49   3.676735      19
50   3.435140      20
51   3.729839      20
52   3.683625       6
53   3.829

On obtient des chiffres dont il faut se mefier car certaines tranche d'age ne possède qu'un seul individu. Ce qui rend la moyenne peu pertinante (même si cet individu à voter pour plusieur film). C'est entre autre le cas des tranches 7, 10 ans etc...

# Question 2 - 10 films les plus similaires à "Star Trek V: The Final Frontier (1989)" selon la mesure du cosinus et de la corrélation

Nous allons entrer nos données dans une matrice creuse de manière à pouvoir faire tout les traitement nécessaire.

In [38]:
from scipy.sparse import coo_matrix, csr_matrix
m = coo_matrix((data.loc[:, 'rating'], (data.loc[:, 'user.id'], data.loc[:, 'item.id'])))
m_ = csr_matrix(m)

## 10 Meilleurs plus proches voision en utilisant cosine

In [39]:
from sklearn.metrics.pairwise import pairwise_distances, cosine_similarity
import numpy as np

star_trek = m.getcol(449)
cosine = cosine_similarity(m.T,star_trek.T).reshape(-1)
indexes = np.argsort(cosine)[-11:][::-1]
print(item.loc[indexes])

     movie id                                    movie title release date  \
449       450         Star Trek V: The Final Frontier (1989)  01-Jan-1989   
229       230           Star Trek IV: The Voyage Home (1986)  01-Jan-1986   
227       228            Star Trek: The Wrath of Khan (1982)  01-Jan-1982   
230       231                          Batman Returns (1992)  01-Jan-1992   
228       229     Star Trek III: The Search for Spock (1984)  01-Jan-1984   
450       451                                  Grease (1978)  01-Jan-1978   
385       386                    Addams Family Values (1993)  01-Jan-1993   
62         63                       Santa Clause, The (1994)  01-Jan-1994   
550       551                       Lord of Illusions (1995)  01-Jan-1995   
380       381                        Muriel's Wedding (1994)  01-Jan-1994   
226       227  Star Trek VI: The Undiscovered Country (1991)  01-Jan-1991   

     video release date                                           IMDb URL 

Nous avons ici gardé les 11 meilleurs résultats de manière a nous assurer de la cohérence de la méthode. En effet il est logique de trouver que le film le plus proche de "Star Trek V: The Final Frontier" est "Star Trek V: The Final Frontier" lui même. C'est une évidence que nous allons tenter de garder en tête tout le long de ce TP.
On peut voir que 4 autres Star Trek apparaissent dans les recommandations, celà peut être un bon indicateur de l'efficacité de cette mesure de distance.

## 10 Meilleurs plus proches voisin en utilisant correlation

Nous allons utiliser la formule de corrélation de Bravais-Pearson suivante :

$\cos(\alpha) = \dfrac{\displaystyle \sum_{i=1}^N (x_i - \bar x)\cdot(y_i - \bar y)}{\sqrt{\displaystyle \sum_{i=1}^N (x_i - \bar x)^2}\cdot\sqrt{\displaystyle \sum_{i=1}^N (y_i - \bar y)^2}}$

adapté de façon matricielle ainsi :

$\cos = \dfrac{startrek\_centred^T * m\_centered}{\sqrt{startrek\_centred^T * startrek\_centred * m\_centred^T * m\_centred}}$

In [57]:
star_trek_centered = star_trek.copy().astype(float)
star_trek_centered[star_trek != 0] = star_trek[star_trek != 0] - np.mean(star_trek[star_trek != 0])

mBin = m_.copy()
mBin.data = mBin.data / mBin.data
m_centered = m_.copy().astype(float)
m_centered[m_ != 0] = csr_matrix(m_ - np.sum(m_, axis=0)/np.sum(mBin,axis=0))[m_ != 0]

corr = star_trek_centered.T.dot(m_centered).toarray().reshape(-1)/np.sqrt(star_trek_centered.T.dot(star_trek_centered).toarray()[0,0]*m_centered.T.dot(m_centered).diagonal())
indexes = np.argsort(corr)
maskedIndexes = indexes[~np.isnan(corr)[indexes]][-11:][::-1]
print(item.loc[maskedIndexes])

      movie id                                movie title release date  \
449        450     Star Trek V: The Final Frontier (1989)  01-Jan-1989   
229        230       Star Trek IV: The Voyage Home (1986)  01-Jan-1986   
227        228        Star Trek: The Wrath of Khan (1982)  01-Jan-1982   
450        451                              Grease (1978)  01-Jan-1978   
230        231                      Batman Returns (1992)  01-Jan-1992   
380        381                    Muriel's Wedding (1994)  01-Jan-1994   
1228      1229                       Poison Ivy II (1995)  01-Jan-1995   
757        758  Lawnmower Man 2: Beyond Cyberspace (1996)  01-Jan-1996   
570        571                    Another Stakeout (1993)  01-Jan-1993   
576        577                           Coneheads (1993)  01-Jan-1993   
1469      1470                    Gumby: The Movie (1995)  01-Jan-1995   

      video release date                                           IMDb URL  \
449                  NaN  http:/

Nous trouvons ici aussi "Star Trek V: The Final Frontier" en première position. Cela valide le fait que nous n'ayons pas fait d'erreur de manipulation de notre matrice creuse et de nos masques.

Pour cette mesure de distance, il est possible de remarqué que la plupart des recommandations sont semblables aux précédentes, seulement deux des autres Star Trek appaissent dans les recomendations. Pour ce film, la mesure de cosine semble plus efficace, ce n'est cependant pas assez pour déterminer quel algorithme est mieux que l'autre. Pour cela il faudrait certainement faire un test A/B avec ces deux distances et comparer les scores Recall@10.

# Question 3 - calcul du vote au film "Star Trek V: The Final Frontier (1989)" des utilisateurs qui n'ont pas de vote pour ce film

Récupération des indices des lignes des utilisateur n'ayant pas voté pour star trek V.

In [41]:
indexes = np.argwhere(m.getcol(449).toarray().reshape(-1)==0).reshape(-1)

Calcul de la valeur du cosinus entre star treck V et les autres film :

In [42]:
cosValues = cosine_similarity(m.T,star_trek.T)

Nous allons maintenant passer au calcul de la distance euclidienne. Nous allons créer un masque nous permettant de ne garder que les valeurs interessantes (cad les films ayant au moins un vote commun avec star trek V) puis faire le calcul des distances euclidiennes.

In [58]:
mBin = m_.copy()
mBin.data = mBin.data / mBin.data
star_trekBin = star_trek.copy()
star_trekBin.data = star_trekBin.data/star_trekBin.data
mask = csr_matrix(star_trekBin.A.repeat([1683],axis=1)).multiply(mBin)
mask = np.array(mask.sum(axis=0)>1).reshape(-1)

#compute the number of common votes between star_trek and the other films
weight = np.sum(csr_matrix(star_trekBin.A.repeat([1683],axis=1)).multiply(mBin),axis=0).reshape(-1)

#the first line following this comment does m_-star_treck giving the différence between star-treck and the other films
#this line is a little longer than that because of the mask usage
euclidianValues  = csr_matrix(star_trekBin.A.repeat([1683],axis=1)).multiply(mBin.multiply((m_-csr_matrix(star_trek.A.repeat([1683],axis=1)))))
euclidianValues.data = euclidianValues.data ** 2
euclidianValues = euclidianValues.sum(axis=0)
euclidianValues.data = np.sqrt(euclidianValues.data)
euclidianValues = np.array(euclidianValues).reshape(-1)

invEuclidian = 1/(euclidianValues+1.0)
invEuclidian = np.multiply(invEuclidian, weight).reshape(-1)
sortedIndexes = np.argsort(invEuclidian)

#does only keep the values of the films that has at least one vote in common with star_trek
maskedIndexes = sortedIndexes[mask[sortedIndexes]]
indexesFilm = np.array(maskedIndexes[0,-21:]).reshape(-1)[::-1]
print(item.loc[indexesFilm])
indexesFilm = indexesFilm[1:]

     movie id                                    movie title release date  \
449       450         Star Trek V: The Final Frontier (1989)  01-Jan-1989   
181       182                              GoodFellas (1990)  01-Jan-1990   
229       230           Star Trek IV: The Voyage Home (1986)  01-Jan-1986   
227       228            Star Trek: The Wrath of Khan (1982)  01-Jan-1982   
89         90            So I Married an Axe Murderer (1993)  01-Jan-1993   
121       122                          Cable Guy, The (1996)  14-Jun-1996   
183       184                        Army of Darkness (1993)  01-Jan-1993   
195       196                      Dead Poets Society (1989)  01-Jan-1989   
79         80                    Hot Shots! Part Deux (1993)  01-Jan-1993   
161       162                          On Golden Pond (1981)  01-Jan-1981   
385       386                    Addams Family Values (1993)  01-Jan-1993   
403       404                               Pinocchio (1940)  01-Jan-1940   

Nous avons dû calculer la distance euclidienne entre Star Trek et les autres films nous-même. Nous avons aussi de façon à garder une certaine cohérence introduit un facteur weight qui représente le nombre de vote commun entre Star Trek V et les autres films de manière à s'assurer que Star Trek V reste bien le plus similaire à lui même.

Passons maintenant au calcul des votes.
Nous allons récuperer les notes des utilisateurs pour les film proches.
Ensuite nous calculons la moyenne des votes pour ces films.
Nous centrons les votes des utilisateurs en leurs soustrayant la moyenne calculée précédement.
Nous appliquons les poids représentés par nos cosValues.
Nous sommons (récuprérant ainsi le numérateur du clacul vu en cours).
Nous divisons par la somme des poids (dénominateur du calcul vu en cours).

Nous ajoutons les valeurs trouvées aux moyennes des votes de chaque utilisateur et trouvons ainsi le vote prédit par notre model.

La fomule utilisée est la suivante:

$predictedVotes = userMean + \frac{noteOfUsersForCloseFilmsCentered * cosValues^T}{\sum_i^n |cosValues_i|} $

In [45]:
noteOfUsersForCloseFilms = m_[list(indexes)].toarray()[:, indexesFilm]
noteOfUsersForCloseFilms = noteOfUsersForCloseFilms.astype(float)
noteOfUsersForCloseFilms[noteOfUsersForCloseFilms == 0] = np.nan


sumFilmsNote = m_[:, indexesFilm].sum(axis=0)
countFilmsNote = (m_[:, indexesFilm] != 0).sum(0)
meanFilmsNote = sumFilmsNote.astype(float) / countFilmsNote


noteOfUsersForCloseFilmsCentered = noteOfUsersForCloseFilms - meanFilmsNote
noteOfUsersForCloseFilmsCenteredWeighted = np.multiply(noteOfUsersForCloseFilmsCentered,cosValues[indexesFilm].T)
noteOfUsersForCloseFilmsCenteredWeightedSumed = np.nansum(noteOfUsersForCloseFilmsCenteredWeighted, axis=1)

k = 1/np.sum(abs(cosValues[indexesFilm]))

noteOfUsersForCloseFilmsCenteredWeightedSumedScaled = noteOfUsersForCloseFilmsCenteredWeightedSumed * k
noteOfUsersForCloseFilmsCenteredWeightedSumedScaled = noteOfUsersForCloseFilmsCenteredWeightedSumedScaled.reshape(-1,1)


usersMeanSum = m_[list(indexes)].sum(axis=1).astype(float)
usersMeanCount = (m_[list(indexes)] != 0).sum(1)
usersMean = usersMeanSum / usersMeanCount
usersMean = usersMean.reshape(-1,1)


votes = usersMean + noteOfUsersForCloseFilmsCenteredWeightedSumedScaled
votes[votes==0] = np.NaN
print(votes)

[[       nan]
 [4.02131966]
 [3.73720565]
 [2.79594352]
 [4.33333333]
 [3.51685234]
 [3.98545435]
 [4.35907599]
 [4.3909884 ]
 [4.52131295]
 [4.04566134]
 [2.66811987]
 [4.82578429]
 [3.04025231]
 [3.74391653]
 [3.55      ]
 [2.82329542]
 [2.6321454 ]
 [4.42939534]
 [4.07870748]
 [2.86315183]
 [3.26374708]
 [3.64505908]
 [3.81327229]
 [3.8263919 ]
 [3.34393862]
 [3.70833333]
 [4.05      ]
 [3.        ]
 [3.8       ]
 [3.94667427]
 [3.48276512]
 [3.77272727]
 [2.88571429]
 [3.64316969]
 [4.22240969]
 [3.96523712]
 [3.69238353]
 [4.13374853]
 [3.64      ]
 [3.94260318]
 [1.92619003]
 [3.54166667]
 [3.64304862]
 [4.31882244]
 [3.84482287]
 [3.78308548]
 [3.26426397]
 [3.66625093]
 [3.56127048]
 [4.42103921]
 [4.49617922]
 [2.95238095]
 [3.1921236 ]
 [2.91781183]
 [3.71334768]
 [3.90059777]
 [3.44522187]
 [3.82454967]
 [2.9491622 ]
 [3.89573826]
 [3.79215637]
 [3.67775704]
 [3.80223794]
 [3.66477272]
 [3.37677892]
 [3.4427186 ]
 [3.55963575]
 [3.38095238]
 [4.00453803]
 [3.71624581]
 [3.35

/Users/baptisterocca/Documents/utc/GI05_Montreal/LOG6308 syst.de.recom/venv/lib/python2.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in divide


Nous obtenons des résultats, il est cependant difficile de savoir si ils sont cohérent. Nous allons donc passer à la question suivante pour quantifier notre erreur.

# Question 4 - calcul du vote au film "Star Trek V: The Final Frontier (1989)" des utilisateurs qui ont voté pour ce film

Seul l'indice des utilisateur à prendre en compte change ici. Nous n'allons donc pas décrire le code à nouveau.

In [46]:
indexes = np.argwhere(m.getcol(449).toarray().reshape(-1)!=0).reshape(-1)

In [47]:
noteOfUsersForCloseFilms = m_[list(indexes)].toarray()[:, indexesFilm]
noteOfUsersForCloseFilms = noteOfUsersForCloseFilms.astype(float)
noteOfUsersForCloseFilms[noteOfUsersForCloseFilms == 0] = np.nan


sumFilmsNote = m_[:, indexesFilm].sum(axis=0)
countFilmsNote = (m_[:, indexesFilm] != 0).sum(0)
moyFilmsNote = sumFilmsNote.astype(float) / countFilmsNote


noteOfUsersForCloseFilmsCentered = noteOfUsersForCloseFilms - moyFilmsNote
noteOfUsersForCloseFilmsCenteredWeighted = np.multiply(noteOfUsersForCloseFilmsCentered,cosValues[indexesFilm].T)
noteOfUsersForCloseFilmsCenteredWeightedSumed = np.nansum(noteOfUsersForCloseFilmsCenteredWeighted, axis=1)
noteOfUsersForCloseFilmsCenteredWeightedSumedScaled = noteOfUsersForCloseFilmsCenteredWeightedSumed/np.sum(abs(cosValues[indexesFilm]))
noteOfUsersForCloseFilmsCenteredWeightedSumedScaled = noteOfUsersForCloseFilmsCenteredWeightedSumedScaled.reshape(-1,1)


usersMeanSum = m_[list(indexes)].sum(axis=1).astype(float)
usersMeanCount = (m_[list(indexes)] != 0).sum(1)
usersMean = usersMeanSum / usersMeanCount
usersMean = usersMean.reshape(-1,1)


predictedVotes = usersMean + noteOfUsersForCloseFilmsCenteredWeightedSumedScaled

Voyons maintenant la performance de notre prédiction, de notre approche Item-Item.

In [48]:
from sklearn.metrics import mean_squared_error as mse

mse(m_[list(indexes)].toarray()[:,449],predictedVotes)

1.1786578568747457

Nous avons une erreur de seulement 1.1787 ce qui semble très bon avec si peu de données et pour un modèle travaillant sur des données aussi sparse. De plus les votes étant de 1 à 5, il est relativement difficile de prédire la note exacte de l'utilisateur, il serait donc plus judicieux de binariser les votes et de prédire si l'utilisateur aimerait ou non le film. C'est certainement pour cette raison que Netflix utilise un vote binaire.

# QUESTION 5 - 10 films conseillés à notre utilisateur

Nous allons reprendre le même code que pour les questions précédente au detail près que nous avons une approche User-User et non Item-Item. Cela ne change pas grand chose (excepté certaine transposition et les axes sur lesquels les sommes sont faites).

In [59]:
warsIndexes = [171, 180]
trekIndexes = [221, 226, 227, 228, 229, 379, 448, 449]

ourUser = csr_matrix(([1,1,5,5,5,5,5,5,5,5],([0]*len(warsIndexes+trekIndexes),warsIndexes+trekIndexes)), shape=(1,1683))

mBin = m_.copy()
mBin.data = mBin.data / mBin.data
ourUserBin = ourUser.copy()
ourUserBin.data = ourUserBin.data/ourUserBin.data
mask = csr_matrix(ourUserBin.A.repeat([944],axis=0)).multiply(mBin)
mask = np.array(mask.sum(axis=1)>1).reshape(-1)


euclidianValues  = csr_matrix(ourUserBin.A.repeat([944],axis=0)).multiply(mBin.multiply((m_-csr_matrix(ourUser.A.repeat([944],axis=0)))))
euclidianValues.data = euclidianValues.data ** 2
euclidianValues = euclidianValues.sum(axis=1)
euclidianValues.data = np.sqrt(euclidianValues.data)
euclidianValues = np.array(euclidianValues).reshape(-1)

weight = np.sum(csr_matrix(ourUserBin.A.repeat([944],axis=0)).multiply(mBin),axis=1).reshape(-1)
invEuclidian = 1/(euclidianValues+1.0)
invEuclidian = np.multiply(invEuclidian, weight).reshape(-1)
maskIndexesNeig = np.argsort(invEuclidian)
maskIndexesNeig = maskIndexesNeig[mask[maskIndexesNeig]]
indexNeig = np.array(maskIndexesNeig[0,-20:]).reshape(-1)[::-1]

cosValues = cosine_similarity(ourUser, m)

noteOfFilmsForCloseUser = m_.toarray()[indexNeig,:]
noteOfFilmsForCloseUser = noteOfFilmsForCloseUser.astype(float)
noteOfFilmsForCloseUser[noteOfFilmsForCloseUser == 0] = np.nan

sumUserNote = m_[indexNeig,:].sum(axis=1)
countUserNote = (m_[indexNeig,:] != 0).sum(1)
moyUserNote = sumUserNote.astype(float) / countUserNote

noteOfFilmsForCloseUserCentered = noteOfFilmsForCloseUser - moyUserNote
noteOfFilmsForCloseUserCenteredWeighted = np.multiply(noteOfFilmsForCloseUserCentered.T,cosValues[0,indexNeig]).T
noteOfFilmsForCloseUserCenteredWeightedSumed = np.nansum(noteOfFilmsForCloseUserCenteredWeighted, axis=0)
noteOfFilmsForCloseUserCenteredWeightedSumedScaled = noteOfFilmsForCloseUserCenteredWeightedSumed/np.sum(abs(cosValues[0,indexNeig]))
noteOfFilmsForCloseUserCenteredWeightedSumedScaled = noteOfFilmsForCloseUserCenteredWeightedSumedScaled.reshape(-1)

FilmMeanSum = m_[indexNeig].sum(axis=0).astype(float)
FilmMeanCount = (m_[indexNeig] != 0).sum(0)
FilmMean = FilmMeanSum / FilmMeanCount
FilmMean = FilmMean.reshape(-1)

votes = FilmMean + noteOfFilmsForCloseUserCenteredWeightedSumedScaled
votes = np.array(votes)[0]
nbNan = np.sum(np.isnan(votes))

filmSorted = np.argsort(votes)
recommendedFilm = filmSorted[-(nbNan + 10):-nbNan][::-1]
print(item.loc[recommendedFilm])

     movie id                                      movie title release date  \
50         51                       Legends of the Fall (1994)  01-Jan-1994   
174       175                                    Brazil (1985)  01-Jan-1985   
172       173                       Princess Bride, The (1987)  01-Jan-1987   
127       128                                  Supercop (1992)  26-Jul-1996   
96         97                        Dances with Wolves (1990)  01-Jan-1990   
12         13                          Mighty Aphrodite (1995)  30-Oct-1995   
114       115  Haunted World of Edward D. Wood Jr., The (1995)  26-Apr-1996   
195       196                        Dead Poets Society (1989)  01-Jan-1989   
56         57                                    Priest (1994)  01-Jan-1994   
208       209                        This Is Spinal Tap (1984)  01-Jan-1984   

     video release date                                           IMDb URL  \
50                  NaN  http://us.imdb.com/M/title-

Nous avons des recommendation qui nous semblent cohérentes dans le sens où les films recommandé semblent avoir en commun des caractéristiques avec star-trek et non star wars (personnages plus dévelopé, histoire moins tournée vers "the heros journey" etc...)

# Question 6 - algorithme bayésien

In [149]:
pro = "engineer"
sexe = "M"
age = 50

voteMin = 4

proRows = user[user["job"]==pro].index.tolist()
sexRows = user[user["gender"]==sexe].index.tolist()
ageRows = user[user["age"] > age-10][user["age"] < age+10].index.tolist()


Probability = []

probJobLike = np.sum(m_[proRows,:] >= voteMin, axis=0).astype(float) / np.sum(m_ >= voteMin, axis=0)
probAgeLike = np.sum(m_[ageRows,:] >= voteMin, axis=0).astype(float) / np.sum(m_ >= voteMin, axis=0)
probSexLike = np.sum(m_[sexRows,:] >= voteMin, axis=0).astype(float) / np.sum(m_ >= voteMin, axis=0)

probaLike = np.sum(m_ >= voteMin, axis=0).astype(float) / np.sum(m_ > 0, axis=0)

proba = np.multiply(probaLike,np.multiply(probJobLike,np.multiply(probAgeLike,probSexLike)))

Probability = np.array(proba)[0]
nbNan = np.sum(np.isnan(Probability))

recomended = np.argsort(Probability)[-(nbNan + 11):-nbNan][::-1]
print(item.loc[recomended])

      movie id                                        movie title  \
1526      1527                                   Senseless (1998)   
1533      1534                                   Twin Town (1997)   
1656      1657                                      Target (1995)   
1448      1449                             Pather Panchali (1955)   
1344      1345  Day the Sun Turned Cold, The (Tianguo niezi) (...   
1125      1126                    Old Man and the Sea, The (1958)   
867        868                            Hearts and Minds (1996)   
1104      1105                                   Firestorm (1998)   
1242      1243                                 Night Flier (1997)   
1368      1369  Forbidden Christ, The (Cristo proibito, Il) (1...   
1202      1203                                     Top Hat (1935)   

     release date  video release date  \
1526  09-Jan-1998                 NaN   
1533  30-May-1997                 NaN   
1656  28-Feb-1996                 NaN   
1448  2

Nous avons dû supprimer une valeurs car l'un des films n'ayant aucun vote (le premier), il renvoie NaN pour nos calcul et donc le sorting le place tout en haut.

Les recommendation nous semblent interessantes car nous avons des classiques et des films d'action que nous imaginons être un corpus interessant pour un ingénieur de 50ans.

# Question 7 - Cross validation

In [183]:
import random
from sklearn.metrics import mean_squared_error as mse

cosValues = cosine_similarity(m.T,star_trek.T)

random.seed(1)

indexes = np.argwhere(m.getcol(449).toarray().reshape(-1)!=0).reshape(-1)

random.shuffle(indexes)
rng = xrange(len(indexes)/5)
underSet = [set(indexes[i::5]) for i in rng]
indexes = set(indexes)
totIndex = set(range(m_.shape[0]))

error = []

for i in rng:
    trainIndex = totIndex - underSet[i]
    testIndex = underSet[i]
    
    mBin = m_[np.array(list(trainIndex)),:].copy()
    mBin.data = mBin.data / mBin.data
    star_trekBin = star_trek.copy()
    star_trekBin.data = star_trekBin.data/star_trekBin.data
    mask = csr_matrix(star_trekBin[np.array(list(trainIndex))].A.repeat([1683],axis=1)).multiply(mBin)
    mask = np.array(mask.sum(axis=0)>1).reshape(-1)

    #compute the number of common votes between star_trek and the other films
    weight = np.sum(csr_matrix(star_trekBin[np.array(list(trainIndex))].A.repeat([1683],axis=1)).multiply(mBin),axis=0).reshape(-1)

    #the first line following this comment does m_-star_treck giving the différence between star-treck and the other films
    #this line is a little longer than that because of the mask usage
    euclidianValues  = csr_matrix(star_trekBin[np.array(list(trainIndex))].A.repeat([1683],axis=1)).multiply(mBin.multiply((m_[np.array(list(trainIndex)),:]-csr_matrix(star_trek[np.array(list(trainIndex))].A.repeat([1683],axis=1)))))
    euclidianValues.data = euclidianValues.data ** 2
    euclidianValues = euclidianValues.sum(axis=0)
    euclidianValues.data = np.sqrt(euclidianValues.data)
    euclidianValues = np.array(euclidianValues).reshape(-1)

    invEuclidian = 1/(euclidianValues+1.0)
    invEuclidian = np.multiply(invEuclidian, weight).reshape(-1)
    sortedIndexes = np.argsort(invEuclidian)

    #does only keep the values of the films that has at least one vote in common with star_trek
    maskedIndexes = sortedIndexes[mask[sortedIndexes]]
    indexesFilm = np.array(maskedIndexes[0,-21:]).reshape(-1)[::-1]
    #print(item.loc[indexesFilm])
    indexesFilm = indexesFilm[1:]
    
    noteOfUsersForCloseFilms = m_[list(testIndex)].toarray()[:, indexesFilm]
    noteOfUsersForCloseFilms = noteOfUsersForCloseFilms.astype(float)
    noteOfUsersForCloseFilms[noteOfUsersForCloseFilms == 0] = np.nan

    sumFilmsNote = m_[:, indexesFilm].sum(axis=0)
    countFilmsNote = (m_[:, indexesFilm] != 0).sum(0)
    moyFilmsNote = sumFilmsNote.astype(float) / countFilmsNote

    noteOfUsersForCloseFilmsCentered = noteOfUsersForCloseFilms - moyFilmsNote
    noteOfUsersForCloseFilmsCenteredWeighted = np.multiply(noteOfUsersForCloseFilmsCentered,cosValues[indexesFilm].T)

    noteOfUsersForCloseFilmsCenteredWeightedSumed = np.nansum(noteOfUsersForCloseFilmsCenteredWeighted, axis=1)
    noteOfUsersForCloseFilmsCenteredWeightedSumedScaled = noteOfUsersForCloseFilmsCenteredWeightedSumed/np.sum(abs(cosValues[indexesFilm]))
    noteOfUsersForCloseFilmsCenteredWeightedSumedScaled = noteOfUsersForCloseFilmsCenteredWeightedSumedScaled.reshape(-1,1)

    usersMeanSum = m_[list(testIndex)].sum(axis=1).astype(float)
    usersMeanCount = (m_[list(testIndex)] != 0).sum(1)
    usersMean = usersMeanSum / usersMeanCount
    usersMean = usersMean.reshape(-1,1)

    predictedVotes = usersMean + noteOfUsersForCloseFilmsCenteredWeightedSumedScaled

    error.append(mse(m_[list(testIndex)].toarray()[:,449],predictedVotes))

print(np.mean(error))
print(np.var(error))

1.1284077928200156
0.028240366966401753


On apprend ici quel sont les films les plus proche avec l'ensemble de train puis sur l'enssemble de validation on calcul les note prévu puis regardons les erreurs. Nous trouvons une très petite variabilité, l'algorithme semble stable donc a peu de biais.

944

(93, 1683)